In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load data
movies = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv')
ratings = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv')

In [2]:
# Merge data
data = pd.merge(ratings, movies, on='book_id')
# print(movies.head())
# print(ratings.head())
print(data.head())

   user_id  book_id  rating  goodreads_book_id  best_book_id  work_id  \
0        1      258       5               1232          1232  3209783   
1       11      258       3               1232          1232  3209783   
2      143      258       4               1232          1232  3209783   
3      242      258       5               1232          1232  3209783   
4      325      258       4               1232          1232  3209783   

   books_count       isbn        isbn13                          authors  ...  \
0          279  143034901  9.780143e+12  Carlos Ruiz Zafón, Lucia Graves  ...   
1          279  143034901  9.780143e+12  Carlos Ruiz Zafón, Lucia Graves  ...   
2          279  143034901  9.780143e+12  Carlos Ruiz Zafón, Lucia Graves  ...   
3          279  143034901  9.780143e+12  Carlos Ruiz Zafón, Lucia Graves  ...   
4          279  143034901  9.780143e+12  Carlos Ruiz Zafón, Lucia Graves  ...   

   ratings_count work_ratings_count work_text_reviews_count ratings_1  \
0

In [3]:
# Create user-item matrix
matrix = data.pivot_table(index='user_id', columns='title', values='rating')

# Remove users who have rated fewer than 50 movies
matrix = matrix.dropna(thresh=50, axis=1).fillna(0)

In [4]:
print(matrix)

title     Angels (Walsh Family, #3)  \
user_id                               
1                               0.0   
2                               0.0   
3                               0.0   
4                               0.0   
5                               0.0   
...                             ...   
53420                           0.0   
53421                           0.0   
53422                           0.0   
53423                           0.0   
53424                           0.0   

title    "حكايات فرغلي المستكاوي "حكايتى مع كفر السحلاوية  #GIRLBOSS  \
user_id                                                                
1                                                     0.0        0.0   
2                                                     0.0        0.0   
3                                                     0.0        0.0   
4                                                     0.0        0.0   
5                                                     0.0   

In [5]:


# Calculate similarities
similarities = cosine_similarity(matrix)

# Get recommendations for user 1
user_id = 1
user_ratings = matrix.loc[user_id].values.reshape(1, -1)
similar_users = similarities[user_id-1]
indices = np.argsort(similar_users)[-10:]
similar_ratings = matrix.iloc[indices].values
weighted_ratings = np.multiply(similar_ratings.T, similar_users[indices]).T
recommendations = weighted_ratings.sum(axis=0) / np.sum(similar_users[indices])

# Print top 5 recommendations
top_indices = np.argsort(recommendations)[-5:]
top_movies = matrix.columns[top_indices]
print("Top recommendations for user", user_id)
for i, title in enumerate(reversed(top_movies)):
    print(i+1, title)

Top recommendations for user 1
1 To Kill a Mockingbird
2 The Kite Runner
3 Life of Pi
4 The Help
5 Memoirs of a Geisha


### Cosine Similarity

In [65]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("movie_dataset.csv")
# df.index = df.index.astype(int)
# df.index


C:\Users\jialo\AppData\Local\Temp\ipykernel_15564\3636132661.py:7: DtypeWarning: Columns (0,1,4,9,13,14,19,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,26

In [66]:
features = ['keywords', 'cast', 'genres', 'director']
for feature in features:
    df[feature] = df[feature].fillna('')

In [67]:
def combined_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']
df["combined_features"] = df.apply(combined_features, axis =1)


In [68]:

cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])
# print("Count Matrix:", count_matrix.toarray())
cosine_sim = cosine_similarity(count_matrix)

# cosine_sim

In [70]:
movie_user_likes = "Dead Poets Society"
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]
movie_index = get_index_from_title(movie_user_likes)

In [71]:
similar_movies = list(enumerate(cosine_sim[movie_index]))

In [72]:
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)
# sorted_similar_movies

In [74]:
def get_title_from_index(index):
    # print(type(index),index)
    return df.loc[index,"title"]
i=0
for movie in sorted_similar_movies:
    print(get_title_from_index(movie[0]))
    i=i+1
    if i>15:
        break

Fur: An Imaginary Portrait of Diane Arbus
Lucky You
Me You and Five Bucks
The Judge
Sherlock Holmes: A Game of Shadows
Zodiac
The Mortal Instruments: City of Bones
Two Girls and a Guy
Grace of Monaco
Radio
Stoker
The Singing Detective
Home for the Holidays
Endless Love
Secret in Their Eyes
The Scarlet Letter


In [52]:
df.loc['206','title']

'Clash of the Titans'